
# Notebook ReAct aplicada

Clase aplicada de ReAct con herramientas simples, trazabilidad y feedback loop de calidad.


In [1]:
from __future__ import annotations

import json
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

# Navigate to 02-prompting root for imports
# Handle multiple execution contexts: direct run, nbconvert, NotebookClient
NOTEBOOK_DIR = Path().resolve()

# Find 02-prompting directory by walking up from current directory
PROMPTING_ROOT = NOTEBOOK_DIR
while PROMPTING_ROOT.name != "02-prompting" and PROMPTING_ROOT.parent != PROMPTING_ROOT:
    PROMPTING_ROOT = PROMPTING_ROOT.parent
    
# If we didn't find it by walking up, try looking for it as a subdirectory
if PROMPTING_ROOT.name != "02-prompting":
    PROMPTING_ROOT = NOTEBOOK_DIR / "02-prompting"
    if not PROMPTING_ROOT.exists():
        # Last resort: we're in project root or similar
        for candidate in [NOTEBOOK_DIR, NOTEBOOK_DIR.parent, NOTEBOOK_DIR.parent.parent]:
            test_path = candidate / "02-prompting"
            if test_path.exists() and (test_path / "common" / "rubrica.py").exists():
                PROMPTING_ROOT = test_path
                break

if str(PROMPTING_ROOT) not in sys.path:
    sys.path.insert(0, str(PROMPTING_ROOT))

from common.rubrica import evaluar_salida

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Falta OPENAI_API_KEY en .env")

model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
client = OpenAI(api_key=api_key)

In [2]:

perfil = {
    "tipo_persona": "consultora de marca personal",
    "gustos": ["arte contemporaneo", "vino", "estrategia de negocio"],
    "estilo": "calida, analitica y directa",
    "contexto": "conversacion retomada despues de varios dias",
}

def analizar_perfil(p):
    return {
        "insights": [
            f"Intereses clave: {', '.join(p.get('gustos', []))}",
            "Conviene una apertura breve con pregunta concreta.",
        ],
        "tono": p.get("estilo", "calido"),
    }

def auditar_respeto(salida):
    texto = f"{salida.get('opener', '')} {salida.get('follow_up', '')}".lower()
    flags = [x for x in ["presion", "insistir", "explicito"] if x in texto]
    return {"ok": len(flags) == 0, "flags": flags}

analysis = analizar_perfil(perfil)
analysis


{'insights': ['Intereses clave: arte contemporaneo, vino, estrategia de negocio',
  'Conviene una apertura breve con pregunta concreta.'],
 'tono': 'calida, analitica y directa'}

In [3]:

prompt_react = f"""
Usa formato ReAct con esta secuencia:
Thought -> Action -> Observation -> Thought -> Action -> Observation -> Final Answer

Herramientas disponibles:
- analizar_perfil: {json.dumps(analysis, ensure_ascii=False)}
- auditar_respeto: se ejecuta despues de proponer mensaje

Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Genera JSON:
{{
  "trace": [
    {{"thought": "...", "action": "...", "observation": "..."}},
    {{"thought": "...", "action": "...", "observation": "..."}}
  ],
  "result": {{
    "opener": "...",
    "follow_up": "...",
    "why_it_works": ["...", "..."]
  }}
}}

No uses emojis.
"""

resp = client.chat.completions.create(
    model=model,
    temperature=0.5,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres un agente ReAct con enfoque profesional y respetuoso."},
        {"role": "user", "content": prompt_react},
    ],
)

react_out = json.loads(resp.choices[0].message.content)
audit = auditar_respeto(react_out["result"])
eval_react = evaluar_salida(perfil, react_out["result"])

print(json.dumps(react_out, ensure_ascii=False, indent=2))
print(json.dumps(audit, ensure_ascii=False, indent=2))
print(json.dumps(eval_react, ensure_ascii=False, indent=2))


{
  "trace": [
    {
      "thought": "La persona ha estado ausente por varios días, así que debo generar un ambiente cálido y acogedor para retomar la conversación.",
      "action": "Proponer una pregunta concreta que invite a la reflexión sobre sus intereses, como el arte contemporáneo o la estrategia de negocio.",
      "observation": "Esto puede ayudar a reactivar el diálogo y mostrar interés genuino en sus pasiones."
    },
    {
      "thought": "Es importante mantener el tono cálido y analítico, mientras se establece un espacio para que la otra persona comparta sus pensamientos.",
      "action": "Formular un mensaje que incluya un saludo cordial y una pregunta sobre su perspectiva actual en relación a sus intereses.",
      "observation": "Esto puede fomentar un intercambio más profundo y significativo."
    }
  ],
  "result": {
    "opener": "Hola, espero que estés bien. Me gustaría saber qué piensas sobre las últimas tendencias en arte contemporáneo.",
    "follow_up": "Adem

In [4]:

prompt_refine = f"""
Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Salida actual:
{json.dumps(react_out, ensure_ascii=False, indent=2)}

Auditoria:
{json.dumps(audit, ensure_ascii=False, indent=2)}

Rubrica:
{json.dumps(eval_react, ensure_ascii=False, indent=2)}

Mejora result.opener y result.follow_up con el mismo estilo ReAct.
No uses emojis.
Devuelve JSON con trace y result.
"""

resp2 = client.chat.completions.create(
    model=model,
    temperature=0.3,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres auditor y optimizador de agentes ReAct."},
        {"role": "user", "content": prompt_refine},
    ],
)

mejorada = json.loads(resp2.choices[0].message.content)
print(json.dumps(mejorada, ensure_ascii=False, indent=2))
print(json.dumps(evaluar_salida(perfil, mejorada["result"]), ensure_ascii=False, indent=2))


{
  "trace": [
    {
      "thought": "La persona ha estado ausente por varios días, así que debo generar un ambiente cálido y acogedor para retomar la conversación.",
      "action": "Proponer una pregunta concreta que invite a la reflexión sobre sus intereses, como el arte contemporáneo o la estrategia de negocio.",
      "observation": "Esto puede ayudar a reactivar el diálogo y mostrar interés genuino en sus pasiones."
    },
    {
      "thought": "Es importante mantener el tono cálido y analítico, mientras se establece un espacio para que la otra persona comparta sus pensamientos.",
      "action": "Formular un mensaje que incluya un saludo cordial y una pregunta sobre su perspectiva actual en relación a sus intereses.",
      "observation": "Esto puede fomentar un intercambio más profundo y significativo."
    },
    {
      "thought": "Debo incluir referencias más específicas a sus gustos para hacer la conversación más atractiva.",
      "action": "Incorporar elementos del arte